First, let's open with pyroot. It should be sufficient to just import TFile

In [2]:
%pylab 
from ROOT import TFile
# import root_numpy
%matplotlib inline

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
simdata = TFile('output_ArCO2_1KeV_10e.root')

ls() prints info on stuff in the root file object I just made

In [4]:
simdata.ls()

TFile**		output_ArCO2_1KeV_10e.root	
 TFile*		output_ArCO2_1KeV_10e.root	
  KEY: TTree	data;1	tree
  KEY: TH1D	NumInts;1	Number of Interactions per event
  KEY: TH1D	NumCorrs;1	Number of Correlated Interactions per event
  KEY: TH1D	MinApproach;1	Minimum possible approach distance between each event generated in the scintillant
  KEY: TH1D	IntDist;1	Distances between interactions


I can access the data tree as an attribute of the root file object. The tree has a Print() method which is handy for summarising what is in the tree:

In [5]:
simdata.data.Print()

******************************************************************************
*Tree    :data      : tree                                                   *
*Entries :       10 : Total =            8951 bytes  File  Size =       3623 *
*        :          : Tree compression factor =   1.16                       *
******************************************************************************
*Br    0 :edep      : vector<double>                                         *
*Entries :       10 : Total  Size=        992 bytes  File Size  =        468 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   1.08     *
*............................................................................*
*Br    1 :x         : vector<double>                                         *
*Entries :       10 : Total  Size=        977 bytes  File Size  =        456 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   1.11     *
*...................................................

One approach is to just convert this to a numpy array directly. You may need to install the root_numpy library. But there's apparently a way to do this in pyroot now, which I haven't used yet (I sent an email or something on Slack a while back explaining the options).

In [6]:
trackarr = root_numpy.tree2array(simdata.data)

In [7]:
trackarr

array([(array([7.64866455e-07, 1.43816754e-04, 1.18393481e-04, 2.70224796e-04,
       1.26098632e-04, 3.40701470e-04]), array([ 0.00000000e+00,  4.16906571e-05,  3.41920251e-03, -8.16743006e-03,
       -4.01341421e-03, -7.10488618e-03]), array([0.00000000e+00, 2.54091114e-05, 1.04637266e-02, 1.12585257e-02,
       9.72191672e-03, 1.08075692e-02]), array([ 0.00000000e+00, -4.66183792e-05, -8.96804413e-03, -7.58186872e-03,
        1.07739479e-03, -3.27734807e-03]), 0., 0., 0.,  0.61758805,  0.37640001, -0.69058528, 0.001),
       (array([0.00079694, 0.00020306]), array([-0.00142848, -0.00671178]), array([0.00426842, 0.01625018]), array([-0.00215016, -0.00752178]), 0., 0., 0., -0.06686276,  0.84160208, -0.53594339, 0.001),
       (array([0.00071743, 0.00028257]), array([0.00365261, 0.00849978]), array([-0.00277604, -0.01294323]), array([-0.0015902 , -0.01016126]), 0., 0., 0.,  0.83510668, -0.4999983 , -0.22934369, 0.001),
       (array([4.94357309e-05, 3.83797018e-04, 3.10158122e-04, 2.29

This is the tree, converted to a numpy array. Separate Geant4 events are stored on the first index.

The array is really a ragged 2d array (because the number of energy deposits in an event isn't a fixed number and some variables stored are just floats).

In [8]:
trackarr['edep']

array([array([7.64866455e-07, 1.43816754e-04, 1.18393481e-04, 2.70224796e-04,
       1.26098632e-04, 3.40701470e-04]),
       array([0.00079694, 0.00020306]), array([0.00071743, 0.00028257]),
       array([4.94357309e-05, 3.83797018e-04, 3.10158122e-04, 2.29133637e-05,
       2.33695765e-04]),
       array([0.0001819 , 0.00021623, 0.00060187]),
       array([5.54028495e-05, 4.68733990e-04, 1.29881894e-04, 3.45981266e-04]),
       array([6.91754730e-05, 1.75281674e-04, 1.61492882e-04, 4.43930454e-04,
       1.50119517e-04]),
       array([0.001]),
       array([0.00053858, 0.00019326, 0.00015699, 0.00011117]),
       array([0.00015101, 0.00051218, 0.00023174, 0.00010507])],
      dtype=object)

Things like the vertex info is just a list.

In [9]:
trackarr['x0']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

The other way to use pyroot is to iterate over the events in the tree.

The loop variable is an 'event' object, and the branches are accessible as attributes.

In [11]:
for evt in simdata.data:
    print(evt.edep)
    print(evt.x0)
    break

{ 7.6486646e-07, 0.00014381675, 0.00011839348, 0.00027022480, 0.00012609863, 0.00034070147 }
0.0


You can fill an array like so (I'll just do edep, x, and x0)

In [22]:
xarr = []
Edeparr = []
x0arr = []
for evt in simdata.data:
    xarr.append(list(evt.x))
    Edeparr.append(list(evt.edep))
    x0arr += [evt.x0]

In [24]:
Edeparr

[[7.648664552132503e-07,
  0.00014381675372089132,
  0.00011839348130768125,
  0.0002702247960894987,
  0.00012609863199936465,
  0.00034070147042735083],
 [0.0007969360853010764, 0.00020306391469892366],
 [0.0007174344731795666, 0.00028256552682043344],
 [4.9435730900700196e-05,
  0.00038379701765728434,
  0.0003101581224280176,
  2.291336374866636e-05,
  0.0002336957652653315],
 [0.0001819048894165275, 0.0002162263989189461, 0.0006018687116645264],
 [5.540284950676223e-05,
  0.00046873399044183276,
  0.00012988189446939855,
  0.00034598126558200645],
 [6.917547303805449e-05,
  0.00017528167402867587,
  0.0001614928819907654,
  0.000443930454417893,
  0.00015011951652461127],
 [0.001],
 [0.000538578983828651,
  0.0001932619598518439,
  0.00015698609573590227,
  0.00011117296058360285],
 [0.00015100952553935557,
  0.0005121799534159,
  0.00023174183583213624,
  0.00010506868521260822]]